# Scaling study for BAI
In this notebook we study the price of fairness for Best arm identification

In [ ]:
import numpy as np
import cvxpy as cp
import matplotlib.pyplot as plt
from algorithms.MAB import MAB
from algorithms.reward_model import LinearRewardModel, EqualGapRewardModel, UniformRewardModel
from algorithms.fairness_model import EqualFairnessModel, CustomFairnessModel, VectorFairnessModel, ProportionalFairnessModel
plt.rc('font', **{'family': 'serif', 'serif': ['Computer Modern']})
plt.rc('text', usetex=True)
SMALL_SIZE = 10
MEDIUM_SIZE = 14
BIGGER_SIZE = 16 

plt.rc('font', size=MEDIUM_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=MEDIUM_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=MEDIUM_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

SEED = 5
np.random.seed(SEED)



def custom_fairness_model(p_val, K):
    def model(theta):
        delta = np.max(theta) - theta
        weight = 1 + np.nan_to_num(1 / np.sqrt(delta), nan=0,posinf=0, neginf=0)
        p = K * p_val * weight / weight.sum()
        return p
    return CustomFairnessModel(K, model = lambda x: model(x))

### Case 1: Linear reward model and pre-specified vector of equal rates
In this case the vector of rewards linearly ranges from $0$ to $1$. The fairness constraints are constant, set to some value $p$

In [ ]:
K_VALUES = [10,20,30]
R_MAX = 1
SOLVER = cp.ECOS
TOL = 1e-6 # to avoid numerical problems


fig, ax = plt.subplots(1, len(K_VALUES), figsize=(14,4.4))

for id_K, K in enumerate(K_VALUES):
    reward_model = LinearRewardModel(R_MAX = R_MAX, K = K)
    p_values = np.logspace(-3, np.log10(1/K - TOL), 100)
    sol_fair_p = np.zeros_like(p_values)
    [w,sol,t] = MAB(reward_model=reward_model).solve_T_star(SOLVER=SOLVER)

    for idx, p_val in enumerate(p_values):
        fairness_model = EqualFairnessModel(K=K, p=p_val)
        instance = MAB(reward_model=reward_model, fairness_model=fairness_model)
        [w_fair, sol_fair, t_fair] = instance.solve_T_star(SOLVER=SOLVER, FAIR=True)
        sol_fair_p[idx] = sol_fair

    delta_sq_min = instance.DELTA_MIN ** 2
    delta_sq_max = instance.DELTA_MAX ** 2
    ax[id_K].plot(p_values, sol_fair_p/sol, 'k--', label=r"$T_{p}^\star /  T^\star$")

    pvalues_plot = np.minimum(K * p_values, 1 - p_values * K)

    ax[id_K].plot(p_values, 1/(1-p_values*K), label=r"$\frac{1}{ 1-p_{\rm sum}}$")
    ax[id_K].grid(which='both',linestyle = ":")
    ax[id_K].set_yscale('log')
    ax[id_K].set_xscale('log')
    ax[id_K].set_title(f'$K$ = {K} - ' + r'$\Delta_{\rm min} = ' + str(np.round(instance.Delta[-2],3)) + r'$ - $\Delta_{\rm max} = ' + str(np.round(instance.Delta.max(),3)) + '$')

    
    ax[id_K].set_xlabel(r'$p$')
    ax[id_K].legend()
ax[0].set_ylabel(r'Scaling characteristic time')
plt.suptitle('Linear reward model with equal pre-specified constraints')
plt.savefig("images/BAI/scaling/scaling_lineargap.pdf", bbox_inches='tight')
plt.show()


### Case 2: Equal gap reward model and pre-specified vector of equal rates
In this case each arm has equal gap of $R_{max}=0.1$. The fairness constraints are constant, set to some value $p$

In [ ]:
K_VALUES = [10,20,30]
R_MAX = 0.1 # The scaling does not depend on this value
SOLVER = cp.ECOS
TOL = 1e-6 # to avoid numerical problems

fig, ax = plt.subplots(1, len(K_VALUES),  figsize=(14,4.4))

for id_K, K in enumerate(K_VALUES):
    reward_model = EqualGapRewardModel(R_MAX = R_MAX, K = K)
    p_values = np.logspace(-3, np.log10(1/K - TOL), 100)
    sol_fair_p = np.zeros_like(p_values)
    [w,sol,t] = MAB(reward_model).solve_T_star(SOLVER=SOLVER)
    

    for idx, p_val in enumerate(p_values):
        fairness_model = EqualFairnessModel(K=K, p=p_val)
        instance = MAB(reward_model, fairness_model=fairness_model)
        [w_fair, sol_fair, t_fair] = instance.solve_T_star(SOLVER=SOLVER, FAIR=True)
        sol_fair_p[idx] = sol_fair

    delta_sq_min = instance.DELTA_MIN ** 2
    delta_sq_max = instance.DELTA_MAX ** 2
    ax[id_K].plot(p_values, sol_fair_p/sol, 'k--', label=r"$T_{p}^\star /  T^\star$")

    pvalues_plot = np.minimum(K * p_values, 1 - p_values * K)

    ax[id_K].plot(p_values, 1/(1-p_values*K), label=r"$\frac{1}{ 1-p_{\rm sum}}$")
    # ax[id_K].plot(p_values, 1/(K*p_values + 1- p_values*K), '-.', label=r"$\frac{1}{\min(K p_{\rm min}, 1-p_{\rm sum})}$"
    ax[id_K].grid(which='both',linestyle = ":")
    ax[id_K].set_yscale('log')
    ax[id_K].set_xscale('log')
    ax[id_K].set_title(f'$K$ = {K} - ' + r'$\Delta_{\rm min} = ' + str(np.round(instance.Delta[-2],3)) + r'$ - $\Delta_{\rm max} = ' + str(np.round(instance.Delta.max(),3)) + '$')

    
    ax[id_K].set_xlabel(r'$p$')
    ax[id_K].legend()

ax[0].set_ylabel(r'Scaling characteristic time')
plt.suptitle('Equal gap model with equal pre-specified constraints')
plt.savefig("images/BAI/scaling/scaling_equalgap.pdf", bbox_inches='tight')
plt.show()


### Case 3: Linear reward model and $\theta$-dependent rates
In this case the reward model is the same as in case 1. The fairness constraints are $\theta$-dependent. In particular, we compute it as follows

$$ p_a(\theta) = pK \frac{f_a(\theta)}{\sum_b f_b(\theta)}, \hbox{ where }f_a(\theta) = 1 + \frac{1}{\Delta_a}$$

with $f_{a^\star} = 1$. This makes the rates inversely proportional to the gap. However, since $f_{a^\star} = 1$, the optimal arm will have a very low fairness constraint, resulting
in the algorithm sampling "good" arms at a very large rate

In [ ]:
K_VALUES = [10,20,30]
R_MAX = 1 # The scaling does not depend on this value
SOLVER = cp.ECOS
TOL = 5e-4 # to avoid numerical problems


fig, ax = plt.subplots(1, len(K_VALUES), figsize=(14,4.4))

for id_K, K in enumerate(K_VALUES):
    reward_model = LinearRewardModel(R_MAX = R_MAX, K = K)
    p_values = np.logspace(-3, np.log10(1/K - TOL), 100)
    sol_fair_p = np.zeros_like(p_values)
    mab = MAB(reward_model=reward_model)
    [w,sol,t] = mab.solve_T_star(SOLVER=SOLVER)

    for idx, p_val in enumerate(p_values):
        instance = MAB(reward_model=reward_model, fairness_model=custom_fairness_model(p_val, K))
        [w_fair, sol_fair, t_fair] = instance.solve_T_star(SOLVER=SOLVER, FAIR=True)
        sol_fair_p[idx] = sol_fair

    ax[id_K].plot(p_values, sol_fair_p/sol, 'k--', label=r"$T_{p}^\star /  T^\star$")

    pvalues_plot = np.minimum(K * p_values, 1 - p_values * K)

    ax[id_K].plot(p_values, 1/(1-p_values*K), label=r"$\frac{1}{ 1-p_{\rm sum}}$")
    ax[id_K].grid(which='both',linestyle = ":")
    ax[id_K].set_yscale('log')
    ax[id_K].set_xscale('log')
    ax[id_K].set_title(f'$K$ = {K} - ' + r'$\Delta_{\rm min} = ' + str(np.round(instance.Delta[-2],3)) + r'$ - $\Delta_{\rm max} = ' + str(np.round(instance.Delta.max(),3)) + '$')

    
    ax[id_K].set_xlabel(r'$p$')
    ax[id_K].legend()
ax[0].set_ylabel(r'Scaling characteristic time')
plt.suptitle(r'Linear reward model with $\theta$-dependent constraints (inversely prop. to the gap)')
plt.savefig("images/BAI/scaling/scaling_lineargap_pdelta.pdf", bbox_inches='tight')
plt.show()


### Case 4: Equal gap reward model and $\theta$-dependent rates
In this case the reward model is the same as in case 2. The fairness constraints are $\theta$-dependent as in case 3.

In [ ]:
K_VALUES = [10,20,30]
R_MAX = 0.1 # The scaling does not depend on this value
SOLVER = cp.ECOS
TOL = 5e-4 # to avoid numerical problems

fig, ax = plt.subplots(1, len(K_VALUES), figsize=(14,4.4))

for id_K, K in enumerate(K_VALUES):
    reward_model = EqualGapRewardModel(R_MAX = R_MAX, K = K)
    p_values = np.logspace(-3, np.log10(1/K - TOL), 100)
    sol_fair_p = np.zeros_like(p_values)
    mab = MAB(reward_model=reward_model)
    [w,sol,t] = mab.solve_T_star(SOLVER=SOLVER)


    for idx, p_val in enumerate(p_values):
        instance = MAB(reward_model=reward_model, fairness_model=custom_fairness_model(p_val, K))
        [w_fair, sol_fair, t_fair] = instance.solve_T_star(SOLVER=SOLVER, FAIR=True)
        sol_fair_p[idx] = sol_fair

    delta_sq_min = instance.DELTA_MIN ** 2
    delta_sq_max = instance.DELTA_MAX ** 2
    ax[id_K].plot(p_values, sol_fair_p/sol, 'k--', label=r"$T_{p}^\star /  T^\star$")

    pvalues_plot = np.minimum(K * p_values, 1 - p_values * K)

    ax[id_K].plot(p_values, 1/(1-p_values*K), label=r"$\frac{1}{ 1-p_{\rm sum}}$")
    #ax[id_K].plot(p_values, 1/pvalues_plot, '-.', label=r"$\frac{1}{\min(K p_{\rm min}, 1-p_{\rm sum})}$")
    ax[id_K].grid(which='both',linestyle = ":")
    ax[id_K].set_yscale('log')
    ax[id_K].set_xscale('log')
    ax[id_K].set_title(f'$K$ = {K} - ' + r'$\Delta_{\rm min} = ' + str(np.round(instance.Delta[-2],3)) + r'$ - $\Delta_{\rm max} = ' + str(np.round(instance.Delta.max(),3)) + '$')

    
    ax[id_K].set_xlabel(r'$p$')
    ax[id_K].legend()
ax[0].set_ylabel(r'Scaling characteristic time')
plt.suptitle(r'Linear reward model with $\theta$-dependent constraints (inversely prop. to the gap)')

plt.savefig("images/BAI/scaling/scaling_equalgap_pdelta.pdf", bbox_inches='tight')
plt.show()


### Case 5: Random gap and $p_a = \lambda w^\star_a$

In [ ]:
K_VALUES = [5]
R_MAX = 10 # The scaling does not depend on this value
SOLVER = cp.ECOS
TOL = 5e-5 # to avoid numerical problems
K = 5
LAMBDA_VALUES = np.linspace(0.001,0.999,100)
SEED = 100
np.random.seed(SEED)

plt.subplots(1,1, figsize=(6,4))
sol_fair_p = np.zeros_like(LAMBDA_VALUES)
sol_p = np.zeros_like(LAMBDA_VALUES)
p_min_vec = np.zeros_like(LAMBDA_VALUES)
p_sum_vec = np.zeros_like(LAMBDA_VALUES)
for id_sim, lamda in enumerate(LAMBDA_VALUES):
    reward_model = EqualGapRewardModel(R_MAX = R_MAX, K = K)
    
    mab = MAB(reward_model=reward_model)
    [w,sol,t] = mab.solve_T_star(SOLVER=SOLVER)
    
    sol_p[id_sim] = sol
    p_val = lamda*w
    #print(p_val)
    #p = custom_fairness_model(mab.Delta, p_val, K)
    p_min_vec[id_sim] = np.min(p_val)
    p_sum_vec[id_sim] = np.sum(p_val)
    fairness_model = VectorFairnessModel(K=K, p = p_val)
    instance = MAB(reward_model=reward_model, fairness_model=fairness_model)
    [w_fair, sol_fair, t_fair] = instance.solve_T_star(SOLVER=SOLVER, FAIR=True)
    sol_fair_p[id_sim] = sol_fair

plt.plot(LAMBDA_VALUES,sol_fair_p/sol_p,label = "$T^\star_p/T^\star$")
plt.plot(LAMBDA_VALUES,1/(K*p_min_vec),label = "$1/K p_{\min}$")
plt.plot(LAMBDA_VALUES,1/(1-p_sum_vec),label = "$1/(1-p_{sum})$")
plt.yscale("log")
plt.legend()
plt.grid(linestyle = ":")
plt.xlabel("$\lambda$")
plt.savefig("images/BAI/scaling/price_fairness.pdf", bbox_inches='tight')

### Case 6: Random gap and $p_a = \lambda w^\star_a$

In [ ]:
R_MAX = 10 # The scaling does not depend on this value
SOLVER = cp.ECOS
TOL = 5e-5 # to avoid numerical problems
K = 5
LAMBDA_VALUES = np.linspace(1e-4,1,100)
SEED = 100
np.random.seed(SEED)

sol_fair_p = np.zeros_like(LAMBDA_VALUES)
sol_p = np.zeros_like(LAMBDA_VALUES)
p_min_vec = np.zeros_like(LAMBDA_VALUES)
p_sum_vec = np.zeros_like(LAMBDA_VALUES)
for id_sim, lamda in enumerate(LAMBDA_VALUES):
    reward_model = EqualGapRewardModel(R_MAX = R_MAX, K = K)
    
    mab = MAB(reward_model=reward_model)
    [w,sol,t] = mab.solve_T_star(SOLVER=SOLVER)
    
    sol_p[id_sim] = sol
    p_val = np.random.rand(K)
    p_val = p_val/np.sum(p_val)/(1+0.1*lamda)

    p_min_vec[id_sim] = np.min(p_val)
    p_sum_vec[id_sim] = np.sum(p_val)
    fairness_model = VectorFairnessModel(K=K, p = p_val)
    instance = MAB(reward_model=reward_model, fairness_model=fairness_model)
    [w_fair, sol_fair, t_fair] = instance.solve_T_star(SOLVER=SOLVER, FAIR=True)
    sol_fair_p[id_sim] = sol_fair

plt.plot(LAMBDA_VALUES,sol_fair_p/sol_p,label = "$T^\star_p/T^\star$")
plt.plot(LAMBDA_VALUES,1/(K*p_min_vec),label = "$1/K p_{\min}$")
plt.plot(LAMBDA_VALUES,1/(1-p_sum_vec),label = "$1/(1-p_{sum})$")
plt.yscale("log")
plt.legend()
plt.xlabel("$\lambda$")
plt.savefig("price_fairness.png",dpi = 600, bbox_inches='tight')

## Case 6: Equal gap case with $p_{a^\star} = 0$ and $p_a = (w_a^\star + 1/K)/2$.

In [ ]:

import cvxpy as cp
plt.rc('font', **{'family': 'serif', 'serif': ['Computer Modern']})
plt.rc('text', usetex=True)
SMALL_SIZE = 10
MEDIUM_SIZE = 16
BIGGER_SIZE = 16

plt.rc('font', size=MEDIUM_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=MEDIUM_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=MEDIUM_SIZE)    # legend fontsize


def solve(K, Delta, p=None):
    w = cp.Variable(K, pos=True)
    problem = cp.max( (cp.inv_pos(w[1:]) + cp.inv_pos(w[0])) / Delta )
    objective = cp.Minimize(problem)
    constraints = [cp.sum(w) == 1]
    if p is not None:
        constraints.append(w >= p)

    problem = cp.Problem(objective, constraints)
    res = problem.solve(solver=cp.ECOS)
    return res, w.value



Kvalues = np.arange(3, 100)
results = []
pmin = []
psum = []
for K in Kvalues:
    Delta = 0.1 * np.ones(K-1)
    p = np.zeros(K)
    T0, w0 = solve(K, Delta, None)


    p = np.zeros(K)#
    p[1:] = (1/K + w0[1:])/2
    Tp, wp = solve(K, Delta, p)
    results.append(Tp/T0)
    pmin.append(np.min(p[1:]))
    psum.append(np.sum(p))

pmin = np.array(pmin)
psum = np.array(psum)



# First plot on the primary y-axis
fig, ax = plt.subplots(1,1, figsize=(6,4))
line1, = plt.plot(Kvalues, results, color='black', linestyle='-', label=r'$T_p^\star/T^\star$')  # Solid black line
plt.text(np.argmax(results)-19, np.max(results)-0.0025, r'$T_p^\star/T^\star$')
plt.ylabel(r'Scaling $T_p^\star/T^\star$')
plt.xlabel('Number of arms $K$')
#plt.title('Price of Fairness: equal gap case')
plt.grid(linestyle = ":")

# Create a secondary y-axis
ax2 = plt.gca().twinx()

# The next two plots on the secondary y-axis
pmin = (1/pmin) / (1 + np.sqrt(Kvalues-1))**2
psum =  (1/(1-psum)) / (1 + np.sqrt(Kvalues-1))**2
line2, = ax2.plot(Kvalues, pmin, label=r'$\frac{1}{p_{\rm min}}$', color='darkred', linestyle='--')  # Dashed dark red line
line3, = ax2.plot(Kvalues, psum, label=r'$\frac{1}{1-p_{\rm sum}}$', color='darkred', linestyle=':')  # Dash-dot red line
ax2.text(25,0.72, r'$\frac{1}{p_{\rm min}}$', color='darkred')
ax2.text(61, 0.26, r'$\frac{1}{1-p_{\rm sum}}$', color='darkred')

# Adding labels to the axes
ax2.set_ylabel(r'Scaling $\frac{1}{p_{\rm min}}, \frac{1}{1-p_{\rm sum}}$', color='darkred')
#ax2.yaxis.label.set_color(line2.get_color())

ax2.spines['right'].set_color('darkred')
# Adding legends
# Combining legends from both axes
lines = [line1, line2, line3]
labels = [line.get_label() for line in lines]
ax2.legend(lines, labels, loc='center right')
plt.savefig("images/BAI/scaling/price_fairness_equalgap.pdf", bbox_inches='tight')



In [ ]:
K = 20
R_MAX = 1 # The scaling does not depend on this value
SOLVER = cp.ECOS
TOL = 6e-5 # to avoid numerical problems

reward_model = LinearRewardModel(R_MAX = R_MAX, K = K)
p_values = np.logspace(-3, np.log10(1/K - TOL), 200)
mab = MAB(reward_model=reward_model)
[w,sol,t] = mab.solve_T_star(SOLVER=SOLVER)
sol_fair_p = np.zeros_like(p_values)
pmin = np.zeros_like(p_values)
psum = np.zeros_like(p_values)


for idx, p_val in enumerate(p_values):
    #p = custom_fairness_model(mab.Delta, p_val, K)
    fairness_model = ProportionalFairnessModel(K, p_val, use_gaps=True)
    instance = MAB(reward_model=reward_model, fairness_model=fairness_model)
    [w_fair, sol_fair, t_fair] = instance.solve_T_star(SOLVER=SOLVER, FAIR=True)
    sol_fair_p[idx] = sol_fair
    p = instance.f(instance.THETA)
    pmin[idx] = np.min(p[p>0])
    psum[idx] = np.sum(p)


fig, ax = plt.subplots(1,1, figsize=(6,4))
line1,= plt.plot(p_values, sol_fair_p / sol, color='black', linestyle='-', label=r'$T_p^\star/T^\star$')
plt.grid(linestyle = ":")
#plt.title("Price of Fairness: larger fairness for suboptimal arms")
#plt.yscale('log')
plt.xscale('log')
plt.xlabel(r'$p_0$')
plt.ylabel(r'Scaling $T_p^\star/T^\star$')


line2, = plt.plot(p_values, 1/(1-psum), label=r'$\frac{1}{1-p_{\rm sum}}$',  color='darkred', linestyle='--')



#ax2 = plt.gca().twinx()

# Adding labels to the axes
#ax2.set_ylabel(r'Scaling $\frac{1}{p_{\rm min}}, \frac{1}{1-p_{\rm sum}}$', color='darkred')

#line3, = plt.plot(p_values, 1/pmin, label=r'$\frac{1}{1-p_{\rm sum}}$',  color='darkred', linestyle=':')
# # Adding legends
# # Combining legends from both axes
# lines = [line1, line2]#, line3]
# ##labels = [line.get_label() for line in lines]
#ax2.legend(lines, labels, loc='upper center')
plt.legend()
plt.savefig("images/BAI/scaling/price_fairness_largerfairness_suboptimalarms.pdf", bbox_inches='tight')
